# TP4_depart
## IFT870/BIN710 Hiver 2021

### Mathieu Lussier (LUSM1202)
### Gabriel Gibeau Sanchez (GIBG2501)

In [92]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
TP4_data_file = "RNA_orignal.csv"
TP4_data = pd.read_csv(TP4_data_file,index_col=0)
TP4_data.shape

(395, 2)

In [94]:
TP4_data[0:10]

,sequence,classe
id,,
AF093014.1/662-809,CACUUUAAGUAAUGGAUGUCUUGGUUCUCAUAACGAUGAAGGACGC...,RF00002
M14649.1/2015-2169,AACUUUCAGCGACGGAUGUCUCGGCUCGAACAACGAUGAAGGGCGC...,RF00002
X53361.2/1206-1368,ACCUGUUGUGGUGGAUGUCUUGGCCCAGGUUCUGAGGAAGGACACA...,RF00002
M86760.1/218-371,AACUUUCAGCAAUGGAUCUCUUGGUUCCCGCGUCGAUGAAGAACGU...,RF00002
X00601.1/3997-4154,AAGCAUAAACGGUGAAUACCUCGACUCCUAAAUCGAUGAAGACCGU...,RF00002
U48228.1/7-166,CAAUCUUAACGAUGGAUGUCUUGGUUCCUAUAGCGAUGAAGGCCGC...,RF00002
U58510.1/2022-2198,AACCCUAGUGAUGGAUGUCUAGGCUCCCGUAUCGAUGAAGAACGUA...,RF00002
AL049755.2/30863-31016,AACUUUCAGCAACGGAUCUCUUGGCUCUCGCAUCGAUGAAGAACGC...,RF00002
U13369.1/6624-6776,GACUCUUAGCGGUGGAUCACUCGGCUCGUGCGUCGAUGAAGAACGC...,RF00002


# Q1
## a)
Un examen des données initiales revèle que les entrées dont la clases est absente contiennent en fait la classe dans la
colonne "sequence", concaténée à la valeur de séquence génétique mais séparée par une virgule. Ce puisque les champs du
fichier sont contenus entre des guillements doubles, mais pour ces entrées un guillemet ouvrant au début de la séquence génétique
ne trouve pas son homologue fermant (il devrait être à la fin de la séquence) ce qui fait en sorte que la virgule est vue
comme un simple caractère de ponctuation, au lieu de servir de séparateur CSV.

Pour remédier à ce problème, nous utilisons les fonctions de la classe $\textit{string}$ et nous sauvegardons le
$\textit{DataFrame}$ corrigé dans un nouveau fichier CSV intitulé "RNA.csv".

In [95]:
# Get indexes for which the entries have no classe
na_indexes = TP4_data['classe'].isna()

data = TP4_data.copy()

# Get missing classes & cleaned sequences
split_vals = TP4_data.loc[na_indexes, 'sequence'].str.split(pat='[,"]', expand=True)
data.loc[na_indexes, 'classe'] = split_vals.iloc[:, 1]
data.loc[na_indexes, 'sequence'] = split_vals.iloc[:, 0]

# Save cleaned data
data.to_csv('RNA_clean.csv')

# We'll free up unused variables to keep our dev space uncluttered
del(split_vals)
del(na_indexes)

## b)

Il y a 12 classes dont la répartition est donnée dans la cellule de code suivante:

In [96]:
# Ceckout unique classes values and instances number for each classe
print(f'Il y a {data.classe.unique().shape[0]} classes')
for ix, cls in enumerate(data.classe.value_counts().index):
    print(f'La classe {cls} contient {data.classe.value_counts()[ix]} entrées')

Il y a 12 classes
La classe RF00008 contient 54 entrées
La classe RF00004 contient 51 entrées
La classe RF00017 contient 49 entrées
La classe RF00002 contient 46 entrées
La classe RF00003 contient 42 entrées
La classe RF00023 contient 36 entrées
La classe RF00015 contient 25 entrées
La classe RF00024 contient 24 entrées
La classe RF00011 contient 19 entrées
La classe RF00012 contient 17 entrées
La classe RF00019 contient 16 entrées
La classe RF00025 contient 16 entrées


## c)

Ici nous balançons le nombre de données en se défaussant des instances excédant le nombre minimal

In [97]:
# Fint out the minimum instances number
classes_rep = data.classe.value_counts()
print(f'Le nombre minimum d\'instance d\'une classe est {classes_rep.min()} pour la classe:\n'
      f'{classes_rep[classes_rep == classes_rep.min()].index.values}')

cut_off = classes_rep.min()

# Trim off exceding instances
for cls in classes_rep.index:
    indexes_to_drop = data[data['classe'] == cls].index
    if indexes_to_drop.shape[0] > cut_off:
        data.drop(index=indexes_to_drop[16:], axis=0, inplace=True)

print(f'Après traitement, les classes contiennent maintenant {cut_off} instance chacune:')
for ix, cls in enumerate(data.classe.value_counts().index):
    print(f'La classe {cls} contient {data.classe.value_counts()[ix]} entrées')

del(classes_rep)
del(cut_off)

Le nombre minimum d'instance d'une classe est 16 pour la classe:
['RF00019' 'RF00025']
Après traitement, les classes contiennent maintenant 16 instance chacune
La classe RF00025 contient 16 entrées
La classe RF00017 contient 16 entrées
La classe RF00023 contient 16 entrées
La classe RF00012 contient 16 entrées
La classe RF00015 contient 16 entrées
La classe RF00019 contient 16 entrées
La classe RF00004 contient 16 entrées
La classe RF00011 contient 16 entrées
La classe RF00024 contient 16 entrées
La classe RF00003 contient 16 entrées
La classe RF00008 contient 16 entrées
La classe RF00002 contient 16 entrées


# Q2
## a)